<a href="https://colab.research.google.com/github/dsfdev2023/studentEvaluatiuon/blob/main/Churn_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Data and Required Packages**

In [41]:
pip install lazypredict


In [43]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier  # Added KNN
from tabulate import tabulate
from lazypredict.Supervised import LazyClassifier
from sklearn.preprocessing import LabelEncoder



In [2]:
df = pd.read_csv("ChurnData.csv")

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,NaN,42,2,0.00,1,1,1,10134888.0,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,11254258.0,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,11393157.0,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,0.0,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,7908410.0,0


In [4]:
df.shape

(10000, 14)

### **EDA**

In [5]:
df.isna().sum()

RowNumber             0
CustomerId            0
Surname               0
CreditScore           0
Geography             0
Gender             1429
Age                   0
Tenure                0
Balance               0
NumOfProducts         0
HasCrCard             0
IsActiveMember        0
EstimatedSalary    1424
Exited                0
dtype: int64

In [7]:
df.isna().sum().sum()

2853

There are 2853 missing values across the dataset

In [6]:
df.duplicated().sum()

0

There are no duplicates values in the data set

# **Input Missing Values**

In [12]:
# Replace missing values with the mean of the respective feature
df.fillna(df.mean(), inplace=True)


In [15]:
df['Gender'].fillna(df['Gender'].mode()[0], inplace=True)


In [16]:
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [17]:
# Display the counts
print(df['Gender'].value_counts())


Male      6086
Female    3914
Name: Gender, dtype: int64


In [19]:
# Distribution of the target variable (churn)
print("\nChurn Distribution:")
print(df['Exited'].value_counts())


Churn Distribution:
0    7963
1    2037
Name: Exited, dtype: int64


# **Find Data Types**

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           8571 non-null   object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  8576 non-null   float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [10]:
df.nunique()

RowNumber          10000
CustomerId         10000
Surname             2932
CreditScore          460
Geography              3
Gender                 2
Age                   70
Tenure                11
Balance             6382
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary     8572
Exited                 2
dtype: int64

In [20]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Male,42,2,0.00,1,1,1,10134888.0,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,11254258.0,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,11393157.0,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,0.0,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,7908410.0,0


# **Split the data into features (X) and target (y)**

In [23]:
X = df.drop(columns=['RowNumber','CustomerId','Surname','Exited'],axis=1)
y = df['Exited']

In [24]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Male,42,2,0.00,1,1,1,10134888.0
1,608,Spain,Female,41,1,83807.86,1,0,1,11254258.0
2,502,France,Female,42,8,159660.80,3,1,0,11393157.0
3,699,France,Female,39,1,0.00,2,0,0,0.0
4,850,Spain,Female,43,2,125510.82,1,1,1,7908410.0


# Create Column Transformer with 3 types of transformers

In [27]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),
    ]
)

In [28]:
X = preprocessor.fit_transform(X)

In [29]:
X.shape

(10000, 13)

# Separate dataset into train and test

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=42)
X_train.shape, X_test.shape

((8000, 13), (2000, 13))

In [44]:
# Initialize and fit the LazyClassifier
clf = LazyClassifier()
models_summary = clf.fit(X_train, X_test, y_train, y_test)

# Display the model comparison results
print(models_summary)

 97%|█████████▋| 28/29 [00:30<00:00,  1.09it/s]

[LightGBM] [Info] Number of positive: 1644, number of negative: 6356
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001952 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 869
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.205500 -> initscore=-1.352267
[LightGBM] [Info] Start training from score -1.352267


100%|██████████| 29/29 [00:30<00:00,  1.05s/it]

(                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
XGBClassifier                      0.87               0.73     0.73      0.86   
AdaBoostClassifier                 0.86               0.72     0.72      0.85   
LGBMClassifier                     0.86               0.72     0.72      0.85   
RandomForestClassifier             0.87               0.72     0.72      0.85   
ExtraTreesClassifier               0.86               0.71     0.71      0.85   
BaggingClassifier                  0.85               0.70     0.70      0.84   
NearestCentroid                    0.71               0.70     0.70      0.74   
GaussianNB                         0.78               0.69     0.69      0.79   
LabelPropagation                   0.81               0.69     0.69      0.81   
LabelSpreading                     0.81               0.69     0.69      0.81   
QuadraticDiscriminantAnalys

In [47]:
# After running LazyPredict as before...

# Extract the DataFrame and sort it by 'Accuracy' in descending order
models_summary_df = models_summary[0]
sorted_models = models_summary_df.sort_values(by='Accuracy', ascending=False)

# Select the top 6 models
top_6_models = sorted_models.head(6)

# Display the top 6 models
print(top_6_models)


                        Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                    
RandomForestClassifier      0.87               0.72     0.72      0.85   
XGBClassifier               0.87               0.73     0.73      0.86   
ExtraTreesClassifier        0.86               0.71     0.71      0.85   
LGBMClassifier              0.86               0.72     0.72      0.85   
AdaBoostClassifier          0.86               0.72     0.72      0.85   
SVC                         0.86               0.68     0.68      0.84   

                        Time Taken  
Model                               
RandomForestClassifier        1.19  
XGBClassifier                 2.85  
ExtraTreesClassifier          1.96  
LGBMClassifier                0.31  
AdaBoostClassifier            0.90  
SVC                           1.91  


In [40]:
# Initialize a list to store model performance results
results = []

# Define a list of classifiers to evaluate
classifiers = [
    ('Random Forest', RandomForestClassifier(random_state=42)),
    ('Logistic Regression', LogisticRegression(random_state=42)),
    ('Support Vector Machine', SVC(random_state=42)),
    ('K-Nearest Neighbors', KNeighborsClassifier()),  # Added KNN
    # Add more classifiers as needed
]

# Iterate through each classifier and evaluate its performance
for clf_name, clf in classifiers:
    clf.fit(X_train, y_train)  # Train the classifier
    y_pred = clf.predict(X_test)  # Make predictions

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Append results to the list
    results.append([clf_name, accuracy, precision, recall, f1])

# Create a pandas DataFrame from the results
results_df = pd.DataFrame(results, columns=['Classifier', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

# Print the results as a table
print(tabulate(results_df, headers='keys', tablefmt='pretty'))

+---+------------------------+----------+--------------------+---------------------+---------------------+
|   |       Classifier       | Accuracy |     Precision      |       Recall        |      F1 Score       |
+---+------------------------+----------+--------------------+---------------------+---------------------+
| 0 |     Random Forest      |  0.8665  | 0.7603305785123967 | 0.4681933842239186  | 0.5795275590551181  |
| 1 |  Logistic Regression   |  0.8145  | 0.5785714285714286 | 0.20610687022900764 | 0.30393996247654786 |
| 2 | Support Vector Machine |  0.8585  | 0.8021978021978022 | 0.37150127226463103 | 0.5078260869565218  |
| 3 |  K-Nearest Neighbors   |  0.8435  | 0.650375939849624  | 0.4402035623409669  | 0.5250379362670714  |
+---+------------------------+----------+--------------------+---------------------+---------------------+
